In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import os
from tqdm import tqdm

In [2]:
image_dir = "E:/Download/val2017"
image_paths = [os.path.join(image_dir, fname) for fname in os.listdir(image_dir) if fname.endswith(".jpg")]

In [3]:
sift = cv2.SIFT_create()
descriptor_list = []
image_descriptors = []

print("🔍 Đang trích xuất SIFT...")
for path in tqdm(image_paths):
    img = cv2.imread(path)
    if img is None:
        continue
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    keypoints, descriptors = sift.detectAndCompute(gray, None)
    if descriptors is not None:
        descriptor_list.extend(descriptors)
        image_descriptors.append((path, descriptors))

🔍 Đang trích xuất SIFT...


100%|██████████████████████████████████████████████████████████████████████████████| 5000/5000 [08:25<00:00,  9.89it/s]


In [ ]:
K = 100  # số cluster (visual words)
kmeans = KMeans(n_clusters=K, random_state=42)
print(" Đang huấn luyện KMeans...")
kmeans.fit(descriptor_list)

🧠 Đang huấn luyện KMeans...


C:\Users\admin\anaconda3\envs\coco_kg\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [ ]:
image_bovw = {}

print("Đang tạo histogram BoVW cho từng ảnh...")
for path, descriptors in image_descriptors:
    labels = kmeans.predict(descriptors)
    hist, _ = np.histogram(labels, bins=np.arange(K+1))
    hist = hist.astype(float)
    hist /= (hist.sum() + 1e-6)  # chuẩn hoá
    image_bovw[path] = hist